In [17]:
from causalnex.inference import InferenceEngine
from causalnex.network import BayesianNetwork
from sklearn.model_selection import train_test_split, KFold
import xml_read
import networkx as nx
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.metrics import log_loss
from var_name import dict_CE
import numpy as np
import warnings
warnings.filterwarnings('ignore')
plt.rcParams['font.sans-serif']=['SimHei']

In [18]:
file_name = 'causal network'
graph_structure = xml_read.parse_xml_sm(file_name+'.xml')
data = pd.read_csv('Structural data_coarse-grained.csv')

In [ ]:
#for the X caused by AMI, calculate the input file containing g and Q, and the g and Q of each covariate

def outcome_k_fold_fit_and_predict(data:pd.DataFrame,  num_folds:int,target:str):
    print(target)
    all_target_probs = np.zeros(len(data))
    kf = KFold(n_splits=num_folds, shuffle=True, random_state=99)
    for fold_idx, (train_idx, test_idx) in enumerate(kf.split(data)):
        train, test = data.iloc[train_idx], data.iloc[test_idx]
        bn = BayesianNetwork(graph_structure)
        bn = bn.fit_node_states(data)
        bn_cpd = bn.fit_cpds(train, method="BayesianEstimator", bayes_prior="K2")
        ie = InferenceEngine(bn_cpd)
        calc_nodes0 = bn_cpd.nodes
        calc_nodes0.remove(target)
        calc_nodes=calc_nodes0#validation A
        data_calc = test.loc[:, calc_nodes]
        list_calc = []
        for idx in range(data_calc.shape[0]):
            row = data_calc.iloc[idx]
            list_calc.append(row.to_dict())   
        marginal_condition = ie.query(list_calc) 
        target_preds = [one_pred[target] for one_pred in marginal_condition]        
        for pred in target_preds:
            all_target_probs[test_idx] = sum(value * prob for value, prob in pred.items())
        print(all_target_probs)
    return all_target_probs
# 
calc_nodes1=['ALB', 'ChestPain', 'Dyspnea', 'HGB', 'Numbness', 'STAB', 'TAB'];
  
for i in range(len(calc_nodes1)):    
    Q=outcome_k_fold_fit_and_predict(data, 10,calc_nodes1[i])
    print(Q)
    combinestr='Input_Q_'+calc_nodes1[i]
    combinestr2='C:/yourdir/AMItoX/input/'+combinestr+'.xlsx'
    df = pd.DataFrame({combinestr: Q})
    df.to_excel(combinestr2, index=False) 

In [ ]:
def treatment_k_fold_fit_and_predict(data:pd.DataFrame,  num_folds:int,target:str):
    print(target)
    all_target_probs = np.zeros(len(data))
    kf = KFold(n_splits=num_folds, shuffle=True, random_state=99)    
    for fold_idx, (train_idx, test_idx) in enumerate(kf.split(data)):
        train, test = data.iloc[train_idx], data.iloc[test_idx]
        bn = BayesianNetwork(graph_structure)
        bn = bn.fit_node_states(data)
        bn_cpd = bn.fit_cpds(train, method="BayesianEstimator", bayes_prior="K2")
        ie = InferenceEngine(bn_cpd)
        calc_nodes0 = bn_cpd.nodes
        calc_nodes2=calc_nodes0.copy()
        calc_nodes2.remove('AMI')
        calc_nodes2.remove(target)
        calc_nodes=calc_nodes2#validation A
        data_calc = test.loc[:, calc_nodes]
        list_calc = []
        for idx in range(data_calc.shape[0]):
            row = data_calc.iloc[idx]
            list_calc.append(row.to_dict())        
        marginal_condition = ie.query(list_calc)            
        target_preds = [one_pred['AMI'] for one_pred in marginal_condition]
        if isinstance(target_preds[0], dict) and 1 in target_preds[0]:
            target_probs = np.array([pred[1] for pred in target_preds])
            all_target_probs[test_idx] = target_probs
        else:
            print(f"Unexpected structure in treatment_preds: {target_preds[0]}")
            break
        print(all_target_probs)
    return all_target_probs
    
calc_nodes1=['ALB', 'ChestPain', 'Dyspnea', 'HGB', 'Numbness', 'STAB', 'TAB'];
for i in range(len(calc_nodes1)):
    g=treatment_k_fold_fit_and_predict(data, 10,calc_nodes1[i])
    print(g)
    combinestr='Input_g_'+calc_nodes1[i]
    combinestr2='C:/yourdir/AMItoX/input/'+combinestr+'.xlsx'
    df = pd.DataFrame({combinestr: g})
    df.to_excel(combinestr2, index=False) 

In [ ]:
def outcome_k_fold_fit_and_predict(data:pd.DataFrame,  num_folds:int,target:str,covariate:str):
    print(target+'_co '+covariate)
    all_target_probs = np.zeros(len(data))
    kf = KFold(n_splits=num_folds, shuffle=True, random_state=99)
    for fold_idx, (train_idx, test_idx) in enumerate(kf.split(data)):
        train, test = data.iloc[train_idx], data.iloc[test_idx]
        bn = BayesianNetwork(graph_structure)
        bn = bn.fit_node_states(data)
        bn_cpd = bn.fit_cpds(train, method="BayesianEstimator", bayes_prior="K2")
        ie = InferenceEngine(bn_cpd)
        calc_nodes0 = bn_cpd.nodes
        calc_nodes0.remove(target)
        calc_nodes0.remove(covariate)
        calc_nodes=calc_nodes0#validation A
        data_calc = test.loc[:, calc_nodes]
        list_calc = []
        for idx in range(data_calc.shape[0]):
            row = data_calc.iloc[idx]
            list_calc.append(row.to_dict())   
        marginal_condition = ie.query(list_calc) 
        target_preds = [one_pred[target] for one_pred in marginal_condition]        
        for pred in target_preds:
            all_target_probs[test_idx] = sum(value * prob for value, prob in pred.items())
        print(all_target_probs)
    return all_target_probs

calc_nodes1=['ALB', 'ChestPain', 'Dyspnea','HGB', 'Numbness', 'STAB', 'TAB'];
calc_nodes=['CVD', 'CFAB', 'Dys', 'Age','CRF', 'Sex','Nutrition','Lifestyle','HTN','ALB', 'ChestPain', 'Dyspnea', 'HGB', 'Numbness', 'STAB', 'TAB']; 
for i in range(len(calc_nodes1)):
    calc_nodes2=calc_nodes.copy()
    calc_nodes2.remove(calc_nodes1[i])
    for j in range(len(calc_nodes2)):       
        Q=outcome_k_fold_fit_and_predict(data, 10,calc_nodes1[i],calc_nodes2[j])
        print(Q)
        combinestr='Q_co'+calc_nodes2[j]
        combinestr2='C:/yourdir/AMItoX/'+calc_nodes1[i]+'/Q_covariate/'+combinestr+'.xlsx'
        df = pd.DataFrame({'Q': Q})
        df.to_excel(combinestr2, index=False) 

In [ ]:
def treatment_k_fold_fit_and_predict(data:pd.DataFrame,  num_folds:int,target:str,covariate:str):
    print(target+'_co '+covariate)
    all_target_probs = np.zeros(len(data))
    kf = KFold(n_splits=num_folds, shuffle=True, random_state=99)    
    for fold_idx, (train_idx, test_idx) in enumerate(kf.split(data)):
        train, test = data.iloc[train_idx], data.iloc[test_idx]
        bn = BayesianNetwork(graph_structure)
        bn = bn.fit_node_states(data)
        bn_cpd = bn.fit_cpds(train, method="BayesianEstimator", bayes_prior="K2")
        ie = InferenceEngine(bn_cpd)
        calc_nodes0 = bn_cpd.nodes
        calc_nodes2=calc_nodes0.copy()
        calc_nodes2.remove('AMI')
        calc_nodes2.remove(target)
        calc_nodes2.remove(covariate)
        calc_nodes=calc_nodes2
        data_calc = test.loc[:, calc_nodes]
        list_calc = []
        for idx in range(data_calc.shape[0]):
            row = data_calc.iloc[idx]
            list_calc.append(row.to_dict())        
        marginal_condition = ie.query(list_calc)           
        target_preds = [one_pred['AMI'] for one_pred in marginal_condition]
        if isinstance(target_preds[0], dict) and 1 in target_preds[0]:
            target_probs = np.array([pred[1] for pred in target_preds])
            all_target_probs[test_idx] = target_probs
        else:
            print(f"Unexpected structure in treatment_preds: {target_preds[0]}")
            break
        print(all_target_probs)
    return all_target_probs

calc_nodes1=['ALB', 'ChestPain', 'Dyspnea', 'HGB', 'Numbness', 'STAB', 'TAB'];
# 马尔可夫边界：EI>0没有CRF，有PCI
calc_nodes=['CVD', 'CFAB', 'Dys', 'Age','CRF', 'Sex','Nutrition','Lifestyle','HTN','ALB', 'ChestPain', 'Dyspnea', 'HGB', 'Numbness', 'STAB', 'TAB'];
for i in range(len(calc_nodes1)):
    calc_nodes2=calc_nodes.copy()
    calc_nodes2.remove(calc_nodes1[i])    
    for j in range(len(calc_nodes2)):
        g=treatment_k_fold_fit_and_predict(data, 10,calc_nodes1[i],calc_nodes2[j])
        print(g)
        combinestr='g_co'+calc_nodes2[j]
        combinestr2='C:/yourdir/AMItoX/'+calc_nodes1[i]+'/g_covariate/'+combinestr+'.xlsx'
        df = pd.DataFrame({'g': g})
        df.to_excel(combinestr2, index=False) 